In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


To install dependencies

Note: After installation, please restart the runtime if it is required.

In [ ]:
!apt update
!apt install -y p7zip-full

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [44.5 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,252 kB]
Get:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,266 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,002 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,342 kB]
Get:13 http://ar

In [ ]:
!pip install av
!pip install filterpy
!pip install onnxruntime_gpu
!pip list

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=78a786501633e19692a17b31673741a92bdcc64b41b266ceb9a80f6464c9187d
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.3 MB/s eta 0:00:00
Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.8.5
aiosignal                        1.3.1
alabaster                        0.7.13
albumentatio

In [ ]:
!cd /content/drive/MyDrive/basketball-broadcaster/models/rtmdet_m && 7z x end2end.onnx.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 11682539 bytes (12 MiB)

Extracting archive: end2end.onnx.zip
--
Path = end2end.onnx.zip
Type = zip
Physical Size = 11682539
Embedded Stub Size = 4
Total Physical Size = 95568619
Multivolume = +
Volume Index = 1
Volumes = 2

  0%      5% - end2end.onnx                    12% - end2end.onnx                    21% - end2end.onnx                    28% - end2end.onnx                    36% - end2end.onnx                    44% - end2end.onnx                    52% - end2end.onnx

The Player & ball tracking with norfair

In [ ]:
import sys
root_path = '/content/drive/MyDrive/basketball-broadcaster'
sys.path.append(root_path)

import os
os.chdir(root_path)

import av
import time
from PIL import Image
import numpy as np
import cv2

from pb_detector import PBDetector
from pb_tracker import PBTracker

input_ = av.open("videos/test1_clip.mp4")
output = av.open("output.mp4", "w")

court_region = [[0, 0], [0, 1080], [1920, 1080], [1920, 0]]
detector = PBDetector(court_region)
tracker = PBTracker()

# Make an output video stream using the input video stream as a template
in_video_stream = input_.streams.video[0]
out_video_stream = output.add_stream(codec_name="h264",
                                      width=in_video_stream.width,
                                      height=in_video_stream.height,
                                      rate=int(in_video_stream.average_rate),
                                      bit_rate=in_video_stream.bit_rate,
                                      pix_fmt=in_video_stream.pix_fmt)

# Make an output audio stream using the input audio stream as a template
in_audio_stream = input_.streams.audio[0]
out_audio_stream = output.add_stream(template=in_audio_stream)

frame_idx = 0
det_cycle = 2

st = time.time()
prec_time = st
decoding_time = 0
inference_time = 0
mot_time = 0
encoder_time = 0

for packet in input_.demux():
    # We need to skip the "flushing" packets that `demux` generates.
    if packet.dts is None:
        continue

    if packet.stream == in_video_stream:
        for frame in packet.decode():
            # To get an input frame
            image = frame.to_image() # PIL image
            rgb_image = np.array(image)
            image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
            current_time = time.time()
            decoding_time += (current_time - prec_time)
            prec_time = current_time

            ##############  To process by AI engine  ############

            # To process the input frame
            player_detections = []
            ball_detections = []
            if frame_idx % det_cycle == 0:
                player_detections, ball_detections = detector.detect(image)
                current_time = time.time()
                inference_time += (current_time - prec_time)
                prec_time = current_time

            player_detections, ball_detections = tracker.run(image, player_detections, ball_detections, period=det_cycle)
            print('Frame {}: detected {} players and {} balls'.format(frame_idx, len(player_detections), len(ball_detections)))
            frame_idx += 1

            # To draw the result
            image = Image.fromarray(rgb_image)
            image = tracker.draw_detections(image, player_detections, ball_detections)
            current_time = time.time()
            mot_time += (current_time - prec_time)
            prec_time = current_time

            ####################################################

            output_frame = av.VideoFrame.from_image(image)
            output_frame.pts = None
            for output_packet in out_video_stream.encode(output_frame):
                # Write the packets to the container
                output_packet.stream = out_video_stream
                output.mux(output_packet)
    elif packet.stream == in_audio_stream:
        packet.stream = out_audio_stream
        output.mux(packet)
        current_time = time.time()
        encoder_time += (current_time - prec_time)
        prec_time = current_time

elapsed = time.time() - st

input_.close()
output.close()

Frame 0: detected 0 players and 0 balls
Frame 1: detected 0 players and 0 balls
Frame 2: detected 9 players and 0 balls
Frame 3: detected 9 players and 0 balls
Frame 4: detected 9 players and 0 balls
Frame 5: detected 9 players and 0 balls
Frame 6: detected 9 players and 0 balls
Frame 7: detected 9 players and 0 balls
Frame 8: detected 9 players and 0 balls
Frame 9: detected 9 players and 0 balls
Frame 10: detected 9 players and 0 balls
Frame 11: detected 9 players and 0 balls
Frame 12: detected 9 players and 0 balls
Frame 13: detected 9 players and 0 balls
Frame 14: detected 9 players and 0 balls
Frame 15: detected 9 players and 0 balls
Frame 16: detected 9 players and 0 balls
Frame 17: detected 9 players and 0 balls
Frame 18: detected 9 players and 0 balls
Frame 19: detected 9 players and 0 balls
Frame 20: detected 9 players and 1 balls
Frame 21: detected 9 players and 1 balls
Frame 22: detected 9 players and 1 balls
Frame 23: detected 9 players and 1 balls
Frame 24: detected 9 playe

KeyboardInterrupt: ignored

In [ ]:
# To print the FPS
print('Processing FPS: ', int(in_video_stream.frames / elapsed))
print('Decoding time: ', int(decoding_time))
print('Inference time: ', int(inference_time))
print('MOT time: ', int(mot_time))
print('Encoder time: ', int(encoder_time))

Processing FPS:  3
Decoding time:  118
Inference time:  61
MOT time:  77
Encoder time:  206
